<a href="https://colab.research.google.com/github/carolvieirav/operalabs/blob/main/freightech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Import

In [ ]:
!pip install selenium

In [ ]:
!apt install chromium-chromedriver

In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select

In [4]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

## class bot

In [100]:
class Bot:
  
  def __init__(self,username,password):
    self.username = username
    self.password = password
    self.bot = webdriver.Chrome('chromedriver',options=options)

  def login(self):
    bot = self.bot
    bot.get("https://freightech.ambev.com.br/#/login")
    bot.implicitly_wait(10) # seconds
    email = bot.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-input[1]/div/div/input")
    password = bot.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-input[2]/div/div/input")
    email.clear()
    password.clear()
    email.send_keys(self.username)
    password.send_keys(self.password)
    bot.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-button[1]/button").click()
    print("Logged in Successfuly")

In [ ]:
automate = Bot("anne.pereira@operalog.com.br", "55026170") 
automate.login()

## dashboard

In [179]:
def meus_chamados(vigencia, chamadas_pen, chamadas_resol, chamadas_aber, pen_alcada):

  ''' 
  Extract dashboard's data for pages "distribuição", "apoio" e "T1" and store 
  in a pandas dataframe.
  '''

  paths = [vigencia, chamadas_aber, chamadas_resol, chamadas_pen, pen_alcada]

  data = []
  for path in paths:
    automate.bot.implicitly_wait(10) # seconds
    elements = automate.bot.find_elements_by_xpath(path)
    
    for elem in elements:
      automate.bot.implicitly_wait(10) # seconds
      data.append(elem.text)
  
  # cleaning data with replace
  data = [item.replace("\n", " ") for item in data]
  data[0] = data[0].replace('Vigência RESTAM ', '')
  data[1] = data[1].replace('Chamados em aberto ','')
  data[1] = data[1].replace(' Chamados', '')
  data[2] = data[2].replace('Chamados resolvidos ','')
  data[2] = data[2].replace(' Chamados', '')
  data[3] = data[3].replace('Chamados pendentes ','')
  data[3] = data[3].replace(' CHAMADOS', '')
  data[3] = data[3].replace(' APROVAR', '')
  data[4] = data[4].replace('Pendentes por alçada', '')
  data[4] = data[4].replace(' CHAMADOS', '')

  # transform data in pandas dataframe
  data = [pd.Series(item) for item in data]
  df = pd.DataFrame(data)
  df = df.T
  df = df.rename(columns={0:'Vigência', 1:"Chamados em aberto", 2:'Chamados resolvidos', 3:'Chamados pendentes', 4: 'Pendentes por alçada'}, inplace = False)
  
  return df

In [181]:
# *bug* tags de distribuição retornam apenas a primeira data

vigencia_dist =  '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[1]/wac-box/div'
chamadas_aber_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[2]/wac-box/div'
chamadas_resol_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[3]/wac-box/div'
chamadas_pen_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[1]/wac-box/div'
pen_alcada_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[2]/wac-box/div'

dist = meus_chamados(vigencia_dist, chamadas_pen_dist, chamadas_resol_dist, chamadas_aber_dist, pen_alcada_dist)
dist

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  if sys.path[0] == '':


,Vigência,Chamados em aberto,Chamados resolvidos,Chamados pendentes,Pendentes por alçada
0,9 DIAS,0 CHAMADOS ROTA 0 AS 0,30 CHAMADOS ROTA 30 AS 0,ROTA 0 AS 0,GOD 0 CAMPO REGIONAL 0 AC PRODUTIVIDADE 0 AC ...


In [172]:
# *bug* tags do apoio retornam dados da aba distibuição

vigencia_apoio =  '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[1]/wac-box/div'
chamadas_aber_apoio = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[2]/wac-box/div'
chamadas_resol_apoio = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[3]/wac-box/div'
chamadas_pen_apoio = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[1]/wac-box/div'
pen_alcada_apoio = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[2]/wac-box/div'

meus_chamados(vigencia_apoio, chamadas_pen_apoio, chamadas_resol_apoio, chamadas_aber_apoio, pen_alcada_apoio)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  if sys.path[0] == '':


,Vigência,Chamados em aberto,Chamados resolvidos,Chamados pendentes,Pendentes por alçada
0,9 DIAS,0 CHAMADOS ROTA 0 AS 0,30 CHAMADOS ROTA 30 AS 0,ROTA 0 AS 0,GOD 0 CAMPO REGIONAL 0 AC PRODUTIVIDADE 0 AC ...


In [167]:
vigencia_t1 =  '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[1]/wac-box/div'
chamadas_aber_t1 = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[2]/wac-box/div'
chamadas_resol_t1 = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[3]/wac-box/div'
chamadas_pen_t1 = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[1]/wac-box/div'
pen_alcada_t1 = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[2]/wac-box/div'

meus_chamados(vigencia_t1, chamadas_pen_t1, chamadas_resol_t1, chamadas_aber_t1, pen_alcada_t1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  if sys.path[0] == '':


,Vigência,Chamados em aberto,Chamados resolvidos,Chamados pendentes,Pendentes por alçada
0,9 DIAS,0 CHAMADOS ROTA 0 AS 0,30 CHAMADOS ROTA 30 AS 0,ROTA 0 AS 0,GOD 0 CAMPO REGIONAL 0 AC PRODUTIVIDADE 0 AC ...


## categoria chamados

In [182]:
automate.bot.implicitly_wait(10) # seconds
automate.bot.get("https://freightech.ambev.com.br/#/categoria-chamados")

In [190]:
table = automate.bot.find_elements_by_xpath('/html/body/app-root/ft-home/div/main/div/ft-categoria-chamados/div/div/div/div')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  """Entry point for launching an IPython kernel.


In [201]:
table_data = []
for item in table:
  table_data.append(item.text)

table_data = [item.replace("\n", " ") for item in table_data]

table_data = table_data[0].strip()
table_data

'Nome Data Criação Usuário Criação 1. Sem evidência 2021-05-13T08:58:05.495 Nathalia Fernanda Viana Rodrigues 2. Evidência incorreta 2021-05-13T08:58:18.443 Nathalia Fernanda Viana Rodrigues 3. Sem evidência de validação 2021-05-13T08:58:36.966 Nathalia Fernanda Viana Rodrigues 4. Sem aprovação prévia do GOD ou Regional via FreighTech 2021-05-13T08:59:09.611 Nathalia Fernanda Viana Rodrigues 5. Ajuste realizado de forma centralizada 2021-05-13T08:59:28.682 Nathalia Fernanda Viana Rodrigues 6. Divergência entre valor solicitado x valor da evidência 2021-05-13T08:59:55.044 Nathalia Fernanda Viana Rodrigues 7. Abertura deve ser feita pelo Sisnec 2021-05-13T09:00:13.265 Nathalia Fernanda Viana Rodrigues 8. Campo incorreto 2021-05-13T09:00:21.916 Nathalia Fernanda Viana Rodrigues 9. Outros 2021-05-14T10:50:23.645 Sergi Blabia Garriga 1 para 9 de 9 Página 1 de 1'

## Exportação

In [240]:
# set implicit wait time
driver.implicitly_wait(10) # seconds

driver.get("https://freightech.ambev.com.br/#/exportacao-dados")
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [242]:
modelo = [item.get_text() for item in soup.find_all('p',{'class': "card-fav card-fav-custom no-fav is-selected"})]
modelo

[]